1. Compare detection with different languages, configs, resolution
Get config impact with varying dpi
Get language impact with varying dpi
Get image enhancement impact with varying dpi, for default and pol language 

In [1]:
import os

In [2]:
os.chdir('./..')
os.getcwd()

'C:\\Users\\Michal\\Documents\\GitHub\\OCR-Receipts'

In [3]:
import cv2
import pytesseract

In [4]:
from scripts.content_detector import process_image
process_image.LOG = False

## Read raw image

In [5]:
os.listdir('images/receipts')

['Paragon_2022-08-11_081037_150dpi.jpg',
 'Paragon_2022-08-11_081131_300dpi.jpg',
 'Paragon_2022-08-11_081332_600dpi.jpg']

In [6]:
raw_img_folderpath = 'images/receipts'

In [7]:
# Get raw images
raw_imgs = dict.fromkeys([150, 300, 600])
for dpi, fname in zip(raw_imgs, os.listdir(raw_img_folderpath)):
    raw_img_filepath = os.path.join(raw_img_folderpath, fname)
    raw_imgs[dpi] = process_image.read_image(raw_img_filepath)

In [ ]:
# Show raw images
for dpi, img in raw_imgs.items():
    #cv2.imshow(f'{dpi} dpi', img)

#cv2.waitKey()

In [8]:
# Get content from raw images
raw_contents = {}
for dpi, img in raw_imgs.items():
    raw_contents[dpi] = pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2RGB),
                                                   config='--psm 4')

In [9]:
# Print the raw content
raw_contents_split = {dpi: content.split('\n') for dpi, content in raw_contents.items()}

for line_150, line_300, line_600 in zip(*raw_contents_split.values()):
    print(f'{line_150:<40}|{line_300:<40}|{line_600:<40}')

a “"CODZIENNIE NISKIE CENY” 4884        |BIEDRONKA “CODZIENNIE NISKIE CENY” 4884 |BIEDRONKA “CODZTENNTE NISKIE CENY” dead 
0 WARSZAWA UL. BOKSERSKA 64             |02-690 WARSZAWA UL, BOKSERSKA 64        |02-690 WARSZAWA UL, BORSERSKA 64        
"FROM MARTINS POLSKA S.A.               |JERONIMO MARTINS POLSKA S.A.            |JERONIMO MARTINS POLSKA 5.A.            
62-025 KOSTRZYN UL.2NINNA 5             |62-025 KOSTRZYN UL. ZNIHNA 5            |62-025 KOSTRZYN UL.ZNIMNA 5             
                                        |                                        |                                        
NIP 7791044327 ar 185679                |NIP 7794011327 wr 485679                |NIP 7794041327 nr 185679                
pipecor FISKALNY                        |PARAGON FISKALNY                        |PARAGON FISKALNY                        
PassataSot tGus700g i x3,99 3,990       |PassataSot thus 708g b { x3,99 3,990    |PassataSot thus /00g [ 1 x3,99 3,990    
Sos Hadero 250al

300 dpi resolution seems to provide the best results.

In [10]:
img = raw_imgs[300]

Now let's compare `image_to_string()` method with different `--psm` arguments.

In [11]:
# Get content from 300 dpi image using `--psm 4` and `--psm 6`
psms = ['--psm 4', '--psm 6']
raw_contents_300 = {
    psm: pytesseract.image_to_string(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), config=psm)
    for psm in psms
}

# Compare the content, omitting empty lines
raw_contents_300_split = {psm: content.split('\n') for psm, content in raw_contents_300.items()}
raw_contents_300_split['--psm 4'] = list(filter(lambda x: x != '', raw_contents_300_split['--psm 4']))
raw_contents_300_split['--psm 6'] = list(filter(lambda x: x != '', raw_contents_300_split['--psm 6']))
for line_psm4, line_psm6 in zip(*raw_contents_300_split.values()):
    print(f'{line_psm4:<50}|{line_psm6:<50}')

BIEDRONKA “CODZIENNIE NISKIE CENY” 4884           |BIEDRONKA “CODZIENNIE NISKIE CENY” 4884           
02-690 WARSZAWA UL, BOKSERSKA 64                  |02-690 WARSZAWA UL. BOKSERSKA 64                  
JERONIMO MARTINS POLSKA S.A.                      |JERONIMO MARTINS POLSKA S.A,                      
62-025 KOSTRZYN UL. ZNIHNA 5                      |62-025 KOSTRZYN UL. ZNIWNA 5                      
NIP 7794011327 wr 485679                          |NIP 7794011327 wr 485679                          
PARAGON FISKALNY                                  |PARAGON FISKALNY                                  
PassataSot thus 708g b { x3,99 3,990              |PassataSot thus700g B { x3,99 3,990               
Sos Madero 250ql 5 { x3,49 3,498                  |Sos Madero 250ql 5 { x3,49 3,498                  
PtNabtyszezFinishd00 = A 1 «13,99 13,998          |PtNabtyszezFinishd00 = A 1 x13,99 13,998          
Sol Kraft 2kg A 4 x5,95 5,95A                     |Sol Kraft 2kg A 4 x5,95 5,95A  

In [12]:
# Get all items that are not equal
diff = []
for line_psm4, line_psm6 in zip(*raw_contents_300_split.values()):
    if line_psm4 != line_psm6:
        diff.append((line_psm4, line_psm6))
        print(f'{line_psm4:<50}|{line_psm6:<50}')

02-690 WARSZAWA UL, BOKSERSKA 64                  |02-690 WARSZAWA UL. BOKSERSKA 64                  
JERONIMO MARTINS POLSKA S.A.                      |JERONIMO MARTINS POLSKA S.A,                      
62-025 KOSTRZYN UL. ZNIHNA 5                      |62-025 KOSTRZYN UL. ZNIWNA 5                      
PassataSot thus 708g b { x3,99 3,990              |PassataSot thus700g B { x3,99 3,990               
PtNabtyszezFinishd00 = A 1 «13,99 13,998          |PtNabtyszezFinishd00 = A 1 x13,99 13,998          
Huszyna-Skarb 1,5( fi 6 x1,59 9,548               |Huszyna-Skarb 1,5( fi 6§ x1,59 9,540              
Woda Nat Laz 1,51 A 6 xt,89 11,348                |Woda Nat Laz 1,51 A 6 «1,89 11,34A                
C2Z0SNEK szt B 2 xf, 79 3,580                     |C2Z0SNEK szt B 2 x1,79 3,58)                      
Pizzalberyjskab00g p { ee 43, 49D                 |Pizzalberyjskab00g p 1 «13,49 13,490              
Pier z seren 400g 0 2 x5, 70 14, -                |Pier z seren 400g 0 2 x5, 70 11

Setting `--psm 6` seems to provide better results.

## Prepare image
Adjust the image to enhance content recognition.

In [13]:
# Get adjusted images based on the raw image
img = raw_imgs[150]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
tresh_from_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
tresh_from_blur = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
cv2.imshow('original', img)
cv2.imshow('gray', gray)
cv2.imshow('blur', blur)
cv2.imshow('tresh from img', tresh_from_img)
cv2.imshow('tresh from blur', tresh_from_blur)
cv2.waitKey()

In [14]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))

opening_from_img = cv2.morphologyEx(tresh_from_img, cv2.MORPH_OPEN, kernel, iterations=1)
invert_from_img = 255 - opening_from_img

opening_from_blur = cv2.morphologyEx(tresh_from_blur, cv2.MORPH_OPEN, kernel, iterations=1)
invert_from_blur = 255 - opening_from_blur

In [ ]:
cv2.imshow('opening from img', opening_from_img)
cv2.imshow('opening from blur', opening_from_blur)
cv2.imshow('invert from img', invert_from_img)
cv2.imshow('invert from blur', invert_from_blur)
cv2.waitKey()

In [15]:
invert_my = 255 - tresh_from_blur
cv2.imshow('invert my', invert_my)
cv2.waitKey()

-1

In [16]:
# Get content from adjusted image
raw_content_proc = pytesseract.image_to_string(cv2.cvtColor(invert_my, cv2.COLOR_BGR2RGB),
                                               config='--psm 4')
raw_content_proc_split = raw_content_proc.split('\n')
raw_content_proc_split = list(filter(lambda x: x != '', raw_content_proc_split))

In [17]:
# Compare it with the original
for line_ori, line_proc in zip(raw_contents_300_split['--psm 4'], raw_content_proc_split):
    if line_ori != line_proc:
        diff.append((line_ori, line_proc))
        print(f'{line_ori:<50}|{line_proc:<50}')

BIEDRONKA “CODZIENNIE NISKIE CENY” 4884           |ae COREE RISKIE CENY” 4884                        
02-690 WARSZAWA UL, BOKSERSKA 64                  |02-698 WARSZAWA UL. BOKSERSKA 61                  
JERONIMO MARTINS POLSKA S.A.                      |JERONING HARTINS POLSKA S.A.                      
62-025 KOSTRZYN UL. ZNIHNA 5                      |62-025 KOSTRZYN Uk. ZNIWNA 5                      
NIP 7794011327 wr 485679                          |HIP 779041327 wt: 185679                          
PassataSot thus 708g b { x3,99 3,990              |PassataSot tGus706g D { x3,99 3,990               
Sos Madero 250ql 5 { x3,49 3,498                  |Sos Haderc 258al_ B 1 x3,49 3,498                 
PtNabtyszezFinishd00 = A 1 «13,99 13,998          |PtWabtyszc2FinisndgO A 4 «13,99 13,990            
Sol Kraft 2kg A 4 x5,95 5,95A                     |Sol Kraft ug i] 1 x5,95 5,958                     
SprayProntoZ50al fl { x8,99 8,99A                 |SprayPronto230al A { x8,99 8,99

## Get content

Check the best image resolution for content extraction.